In [1]:
from PyBTLS.py.bridge.bridge import Bridge
from PyBTLS.py.bridge.flow import LaneFlow, BridgeFlow
from PyBTLS.py.traffic.constant_vehicle import ConstantVehicle
from PyBTLS.py.traffic.garage import Garage, Kernels
import PyBTLS
import numpy as np

***Current In Development Workflow:***
1.    Define IL, Bridge, and traffic (Lane flow, garage, Grave folder, etc)
2.    Execute write_as_txt() for all these files
3.    Set up PyBTLS.ConfigData() object to point to where the txt are saved + other options
4.    PyBTLS.run()
5.    Execute read_outputs() function

In [2]:
# 1. Define IL, Bridge, and traffic (Lane flow, garage, Grave folder, etc)

# Define the bridge
# This is similar to PyCBA syntax
L = [50.0] #m
R = [-1, 0, -1, 0] #Simply supported
bridge = Bridge(L=L, R=R, n_lane=1)

# Define the Load Effects to analyse
# Adding load effects can be done in two ways
# The first way is to use BTLS built-in influence line
# e.g., this is Influence Line 1, which computes moment at midspan
# See BTLS manual for full list of built-in load effects
bridge.add_load_effect(load_effect = 1) 
# Another way to define load effects is by specifiying:
#   load_effect ('M', 'V', or 'R' for moment, shear or reaction)
#   poi         (The point where the load effect is to be measured)
#   max_error   (The maximum error of the IL calculation. Error is relative to actual values.)
# Some other available arguments are:
#   lane_factor (The lane_factor. Can be a list for different factors for each lane, or a singular value for same lane_factor for all. default 1.0)
#   POT         (Minimum LE to be included in Peak Over Threshold (POT) analysis. default to 0.0)
bridge.add_load_effect(load_effect = "M", poi = 3/4 * L[0], max_error = 0.001)
bridge.add_load_effect(load_effect = "M", poi = 1/4 * L[0], max_error = 0.001)
bridge.add_load_effect(load_effect = "V", poi = 1/4 * L[0], max_error = 0.001)

# Defining the lane flow
# Lane flow can be constructed using:
#   - data frame (NOT recommended as it is prone to error)
#   - csv file
#   - default files
# Construction via data frame is done using the Pandas.DataFrame arguments and syntax. See Pandas documentation for details.
# Construction via csv file require you to supply the following arguments:
#   path        (Path to the csv file, in string)
# Construction via the default files can be done using the load_default() method. The method can take the argument:
#   sample      (The sample number, either 1 or 2. Currently there are two default lane flow definitions available. Default to 1)
lane_1 = LaneFlow()
lane_1.load_default()


# Define the Bridge Flows
# Multiple lanes can be created. This is then added to a BridgeFlow object, which describe the flow of all the lanes in the bridge
# Lane can be added at initialisation as list or BtlsFlow object, or after initialisation via:
#   - add() method or other aliases of the method: add_lane_flow(), append(), extend())
#   - insert(idx, lane_flow) : insert new lane flow into the array
#   - list indexing          : This is similar to regular list indexing, e.g. bridge_flow[1] = lane_2, del bridge_flow[0], etc.
bridge_flow = BridgeFlow(lane_1)

# Import traffic files
constant_vehicle = ConstantVehicle()
constant_vehicle.load_default()

garage = Garage()
garage.load_default()

kernels = Kernels()
kernels.load_default()

from PyBTLS.py.traffic.PyBtlsDefaultTraffic import Auxerre #OLD CODE, BUT NO REPLACEMENT YET
grave_traffic = Auxerre #OLD CODE, BUT NO REPLACEMENT YET

In [3]:
# 2. Execute write_as_txt() or write_as_csv() for all the files

# Write as txt/csv all inputs
bridge_path = "Input/bridge.txt"
bridge_flow_path = "Input/lane_flow.csv"
il_path = "Input/IL.txt"
#constant_vehicle_path = "Input/constant_vehicle.csv"
#garage_path = "Input/garage.txt"
#kernels_path = "Input/kernels.csv"
grave_traffic_path = "Input/Traffic"

bridge.write_as_txt(il_path = il_path, bridge_path = bridge_path)
#constant_vehicle.write_as_csv(path = constant_vehicle_path)
#garage.write_as_txt(path = garage_path)
#kernels.write_as_csv(path = kernels_path)
grave_traffic.write_as_csv(path = grave_traffic_path)

bridge_flow._check_lane_clashes()
bridge_flow._check_lane_complete()
#Check that the number of lanes in the bridge_flow is equal to the number of lanes in the bridge
if not len(bridge_flow.lane_flow) == bridge.n_lane:
    ValueError("ERROR: Number of lanes in BridgeFlow is not equal to the number of lanes in Bridge")
bridge_flow.write_as_csv(path = bridge_flow_path)

In [4]:
# 3. Set up PyBTLS.ConfigData() object to point to where the txt are saved + other options

config = PyBTLS.ConfigData()

config.veh_gen_grave(
        lanes_file=bridge_flow_path,
        traffic_folder=grave_traffic_path,
        no_days=10,
        truck_track_width=190.0,
        lane_eccentricity_std=0.0,
    )

"""
config.veh_gen_nominal(
    lanes_file=bridge_flow_path, 
    constant_file=constant_vehicle_path, 
    no_days=10, 
    lane_eccentricity_std=0.0)
"""

"""
config.veh_gen_garage(
    lanes_file=bridge_flow_path, 
    garage_file=garage_path, 
    file_format=4, 
    kernel_file="kernels_path", 
    no_days=10, 
    lane_eccentricity_std=20)
"""

config.flow_gen_NHM(
    vehicle_classification=1, 
    traffic_folder=grave_traffic_path)

# config.flow_gen_constant(vehicle_classification=1, constant_speed=36, constant_gap=10)

# config.flow_gen_congestion(vehicle_classification=1, congested_spacing=26.1, congested_speed=49.7, congested_gap_coef_var=0.05)

# config.flow_gen_freeflow(vehicle_classification=1)

config.simulation(
    bridge_file=bridge_path,
    infline_file=il_path,
    infsurf_file="./IS.txt",
    calc_time_step=0.1,
    min_gvw=35,
)

# config.output_general()


config.output_BM(
    write_blockmax=True, 
    write_summary=True,
    write_vehicle=True,
    write_mixed=True,
    block_size_days=int(1)
    )

# config.output_POT(write_pot=True)

config.output_vehicle_file(
    write_vehicle_file=True,
    vehicle_file_format=4,
    write_flow_stats=True,
    buffer_size=100000)

config.output_stats(
    write_stats=True,
    write_cumulative=True,
    write_intervals=True,
    interval_size=int(24*3600)
    )

# config.output_fatigue(do_rainflow=True)

PyBTLS.run()

Starting simulation...
Day complete...
Reading influence line file: Input/IL.txt
Reading influence surface file: ./IS.txt


*** Warning: Influence Surface file could not be opened



Flushing buffer of 100000 vehicles at 10/1/0 18:33:13.0232
	1	2	3	4	5	6	7	8	9	10

Flushing buffer of 2601 vehicles at 11/1/0 0:0:2.98241


# Analysis
The output of the analysis is store in btls.output object For example, if we're interested in the block maxima seperated by number of vehicles in a crossing:

In [5]:
from PyBTLS.py.output.output_reader import read_interval_statistics_files
from PyBTLS.py.output.output_reader import read_cumulative_statistics_file
from PyBTLS.py.output.output_reader import read_block_maxima_summary_file
from PyBTLS.py.output.output_reader import read_block_maxima_event_separated_vehicles_file
from PyBTLS.py.output.output_reader import read_block_maxima_event_mixed_vehicles_file

In [6]:
bm_event = read_block_maxima_event_separated_vehicles_file(froot="./", bridge_length=np.sum(bridge.L), file_format="MON")
display(bm_event)
for i in bm_event:
    display(i)

[20 x BlockMaximaEvent object at 0x7eff6d022110,
 40 x BlockMaximaEvent object at 0x7eff6d022050,
 40 x BlockMaximaEvent object at 0x7eff6d0aceb0]

,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,3,1,5632.6,248260.7,60.20,3,3 vehicle(s) Vehicle object at 0x7eff6d0227a0
1,3,2,3616.2,248260.3,50.08,3,3 vehicle(s) Vehicle object at 0x7eff6d022b30
2,3,3,4297.4,248260.3,50.08,3,3 vehicle(s) Vehicle object at 0x7eff6d023700
3,3,4,323.8,248260.4,52.61,3,3 vehicle(s) Vehicle object at 0x7eff6d023bb0
4,5,1,2682.2,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d023c10
5,5,2,1825.3,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d022230
6,5,3,1576.8,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d022d40
7,5,4,126.1,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d022a70
8,6,1,3518.9,454030.3,59.59,3,3 vehicle(s) Vehicle object at 0x7eff6d0ac340
9,6,2,1837.6,454030.3,59.59,3,3 vehicle(s) Vehicle object at 0x7eff6d0ac400


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,7296.6,20933.8,42.96,2,2 vehicle(s) Vehicle object at 0x7eff6d1cc070
1,1,2,5795.1,20933.9,45.45,2,2 vehicle(s) Vehicle object at 0x7eff6d1ccb80
2,1,3,5710.5,20933.7,40.48,2,2 vehicle(s) Vehicle object at 0x7eff6d1cd120
3,1,4,394.8,43776.5,46.87,2,2 vehicle(s) Vehicle object at 0x7eff6d1ccf10
4,2,1,8870.4,116798.8,37.11,2,2 vehicle(s) Vehicle object at 0x7eff6d1ccfd0
5,2,2,6826.5,116799.2,46.70,2,2 vehicle(s) Vehicle object at 0x7eff6d1cd090
6,2,3,6697.9,116798.8,37.11,2,2 vehicle(s) Vehicle object at 0x7eff6d1cd150
7,2,4,484.5,116799.0,41.91,2,2 vehicle(s) Vehicle object at 0x7eff6d1cd210
8,3,1,8092.8,218510.8,36.16,2,2 vehicle(s) Vehicle object at 0x7eff6d1cd2d0
9,3,2,6269.2,218511.2,46.06,2,2 vehicle(s) Vehicle object at 0x7eff6d1cd390


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,7444.2,80328.4,33.61,1,1 vehicle(s) Vehicle object at 0x7eff6d259510
1,1,2,5591.8,80328.7,40.81,1,1 vehicle(s) Vehicle object at 0x7eff6d259720
2,1,3,5784.7,80327.9,21.61,1,1 vehicle(s) Vehicle object at 0x7eff6d258730
3,1,4,440.4,80328.0,24.01,1,1 vehicle(s) Vehicle object at 0x7eff6d259870
4,2,1,6998.5,133064.5,33.82,1,1 vehicle(s) Vehicle object at 0x7eff6d259930
5,2,2,5237.8,149308.8,39.28,1,1 vehicle(s) Vehicle object at 0x7eff6d2599f0
6,2,3,5406.2,133064.0,21.74,1,1 vehicle(s) Vehicle object at 0x7eff6d259ab0
7,2,4,425.4,149308.2,23.57,1,1 vehicle(s) Vehicle object at 0x7eff6d259b70
8,3,1,7028.0,229799.2,33.56,1,1 vehicle(s) Vehicle object at 0x7eff6d259c30
9,3,2,5234.8,229799.5,41.30,1,1 vehicle(s) Vehicle object at 0x7eff6d259cf0


Suppose we want to see:  
for a 3 vehicles crossing event, the vehicles that creates the most extreme load_effect for Load Effect 1. We want to see their headway at the maximum loading event.  
We can use regular pandas expressions and methods.

In [7]:
# Shorthand for the output object we're interested in
df = bm_event[0]
# Filter only load_effect 1
df[df['load_effect_num'] == 1]
# Get the critical vehicles
critical_platoon = df["vehicles"][np.argmax(df.max_load_effect)]
#Calculate their headways
arrival_time_in_second = (critical_platoon.second + critical_platoon.minute * 60 + critical_platoon.hour * 3600).to_numpy()
headway = arrival_time_in_second[1:] - arrival_time_in_second[0:-1]

# Display the results
print("Headways of the critical vehicles for 3 vehicles crossing event are:", headway, "seconds")
display(critical_platoon)

Headways of the critical vehicles for 3 vehicles crossing event are: [1.175 0.803] seconds


,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,3,1,2010,20,57,38.280,5,0,43256.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,3,1,2010,20,57,39.455,5,0,44950.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,3,1,2010,20,57,40.258,4,0,38000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Other Outputs
Current available outputs that can be imported to Python

In [8]:
read_cumulative_statistics_file(froot = "./", bridge_length=np.sum(bridge.L))

,load_effect,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,110015,117584,117584,5.03,8870.37,3426.73,1529.69,2339955.11,-0.36,-0.87,102454.0,7553.0,8.0
1,2,110015,117584,117584,7.54,6826.49,2598.60,1147.25,1316191.57,-0.37,-0.88,102454.0,7553.0,8.0
2,3,110015,117584,117584,2.51,6697.86,2638.49,1185.91,1406390.04,-0.34,-0.89,102454.0,7553.0,8.0
3,4,110015,117584,117584,-69.04,493.80,200.43,92.78,8608.19,-0.35,-0.85,102454.0,7553.0,8.0


In [9]:
interval_statistics = read_interval_statistics_files(froot = "./", bridge_length=np.sum(bridge.L))
display(interval_statistics)
for i in interval_statistics:
    display(i)

[Load Effect 3:  9 row(s) of IntervalStatistics object at 0x7eff6d21fc40,
 Load Effect 1:  9 row(s) of IntervalStatistics object at 0x7eff6d21fb50,
 Load Effect 2:  9 row(s) of IntervalStatistics object at 0x7eff6d21eb90,
 Load Effect 4:  9 row(s) of IntervalStatistics object at 0x7eff6d25be20]

,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11109,11911,11911,3.51,5784.72,2637.88,1178.13,1387986.00,-0.35,-0.88,10307.0,802.0,0.0
1,2,172800.0,10965,11715,11715,9.22,6697.86,2642.42,1192.33,1421651.48,-0.35,-0.89,10215.0,750.0,0.0
2,3,259200.0,10824,11490,11490,9.09,6269.57,2663.21,1178.01,1387704.20,-0.37,-0.85,10159.0,664.0,1.0
3,4,345600.0,11094,11853,11853,2.51,5618.25,2629.06,1188.32,1412095.52,-0.34,-0.91,10335.0,759.0,0.0
4,5,432000.0,10777,11495,11495,5.87,6139.29,2643.90,1191.08,1418665.30,-0.37,-0.89,10060.0,716.0,1.0
5,6,518400.0,10911,11627,11627,4.96,6640.59,2628.74,1185.46,1405320.98,-0.33,-0.90,10197.0,712.0,2.0
6,7,604800.0,11005,11770,11770,4.99,5949.72,2619.73,1187.63,1410458.41,-0.32,-0.93,10241.0,763.0,1.0
7,8,691200.0,11213,12025,12025,5.16,5612.24,2655.20,1190.65,1417657.73,-0.34,-0.88,10401.0,812.0,0.0
8,9,777600.0,10997,11775,11775,6.08,5836.76,2643.84,1184.61,1403304.99,-0.35,-0.90,10219.0,778.0,0.0


,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11109,11911,11911,7.02,7444.24,3423.06,1522.15,2316933.31,-0.36,-0.87,10307.0,802.0,0.0
1,2,172800.0,10965,11715,11715,18.44,8870.37,3431.70,1537.63,2364300.49,-0.37,-0.87,10215.0,750.0,0.0
2,3,259200.0,10824,11490,11490,18.18,8092.79,3457.33,1520.24,2311144.27,-0.39,-0.84,10159.0,664.0,1.0
3,4,345600.0,11094,11853,11853,5.03,7619.53,3414.13,1532.78,2349406.86,-0.36,-0.89,10335.0,759.0,0.0
4,5,432000.0,10777,11495,11495,11.73,7642.03,3432.71,1536.07,2359513.82,-0.38,-0.88,10060.0,716.0,1.0
5,6,518400.0,10911,11627,11627,9.93,8229.11,3415.95,1528.26,2335586.00,-0.34,-0.88,10197.0,712.0,2.0
6,7,604800.0,11005,11770,11770,9.97,7346.95,3400.98,1530.88,2343583.42,-0.34,-0.91,10241.0,763.0,1.0
7,8,691200.0,11213,12025,12025,10.32,7435.02,3450.43,1536.31,2360242.19,-0.36,-0.86,10401.0,812.0,0.0
8,9,777600.0,10997,11775,11775,12.16,7594.35,3435.37,1526.72,2330875.36,-0.37,-0.87,10219.0,778.0,0.0


,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11109,11911,11911,10.52,5795.13,2597.00,1141.23,1302397.37,-0.38,-0.88,10307.0,802.0,0.0
1,2,172800.0,10965,11715,11715,27.66,6826.49,2606.69,1152.83,1329012.46,-0.39,-0.87,10215.0,750.0,0.0
2,3,259200.0,10824,11490,11490,27.27,6269.22,2619.13,1141.17,1302278.10,-0.40,-0.85,10159.0,664.0,1.0
3,4,345600.0,11094,11853,11853,7.54,5848.08,2586.59,1149.49,1321318.33,-0.36,-0.91,10335.0,759.0,0.0
4,5,432000.0,10777,11495,11495,17.60,5762.38,2604.24,1153.36,1330241.59,-0.39,-0.88,10060.0,716.0,1.0
5,6,518400.0,10911,11627,11627,14.89,6695.83,2589.76,1148.06,1318031.74,-0.36,-0.89,10197.0,712.0,2.0
6,7,604800.0,11005,11770,11770,14.96,5572.00,2580.19,1149.50,1321354.68,-0.34,-0.93,10241.0,763.0,1.0
7,8,691200.0,11213,12025,12025,15.47,5771.85,2615.24,1151.71,1326442.23,-0.37,-0.87,10401.0,812.0,0.0
8,9,777600.0,10997,11775,11775,18.24,5672.31,2605.13,1141.98,1304121.84,-0.37,-0.89,10219.0,778.0,0.0


,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11109,11911,11911,-56.40,440.45,199.98,92.54,8563.74,-0.36,-0.84,10307.0,802.0,0.0
1,2,172800.0,10965,11715,11715,-68.79,484.49,200.68,93.32,8708.76,-0.36,-0.85,10215.0,750.0,0.0
2,3,259200.0,10824,11490,11490,-59.78,442.58,202.50,91.94,8453.78,-0.38,-0.82,10159.0,664.0,1.0
3,4,345600.0,11094,11853,11853,-69.04,420.26,199.70,92.89,8628.61,-0.34,-0.86,10335.0,759.0,0.0
4,5,432000.0,10777,11495,11495,-58.52,457.83,201.02,93.18,8682.89,-0.37,-0.87,10060.0,716.0,1.0
5,6,518400.0,10911,11627,11627,-53.70,493.80,199.98,92.57,8570.10,-0.33,-0.87,10197.0,712.0,2.0
6,7,604800.0,11005,11770,11770,-68.64,437.38,199.00,93.00,8649.54,-0.34,-0.88,10241.0,763.0,1.0
7,8,691200.0,11213,12025,12025,-64.57,445.55,201.53,93.22,8690.41,-0.35,-0.83,10401.0,812.0,0.0
8,9,777600.0,10997,11775,11775,-60.47,440.41,200.90,92.66,8585.99,-0.36,-0.85,10219.0,778.0,0.0


In [10]:
bm_event = read_block_maxima_event_separated_vehicles_file(froot="./", bridge_length=np.sum(bridge.L), file_format="MON")
display(bm_event)
for i in bm_event:
    display(i)

[20 x BlockMaximaEvent object at 0x7eff6d0236a0,
 40 x BlockMaximaEvent object at 0x7eff6d25b820,
 40 x BlockMaximaEvent object at 0x7eff6d0233d0]

,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,3,1,5632.6,248260.7,60.20,3,3 vehicle(s) Vehicle object at 0x7eff6d08e950
1,3,2,3616.2,248260.3,50.08,3,3 vehicle(s) Vehicle object at 0x7eff6d08ea10
2,3,3,4297.4,248260.3,50.08,3,3 vehicle(s) Vehicle object at 0x7eff6d08eb30
3,3,4,323.8,248260.4,52.61,3,3 vehicle(s) Vehicle object at 0x7eff6d08e9e0
4,5,1,2682.2,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d08eaa0
5,5,2,1825.3,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d08eb60
6,5,3,1576.8,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d08ec20
7,5,4,126.1,358519.3,56.85,3,3 vehicle(s) Vehicle object at 0x7eff6d08ece0
8,6,1,3518.9,454030.3,59.59,3,3 vehicle(s) Vehicle object at 0x7eff6d08eda0
9,6,2,1837.6,454030.3,59.59,3,3 vehicle(s) Vehicle object at 0x7eff6d08ee60


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,7296.6,20933.8,42.96,2,2 vehicle(s) Vehicle object at 0x7eff6d0c3490
1,1,2,5795.1,20933.9,45.45,2,2 vehicle(s) Vehicle object at 0x7eff6d0c3550
2,1,3,5710.5,20933.7,40.48,2,2 vehicle(s) Vehicle object at 0x7eff6d0c35b0
3,1,4,394.8,43776.5,46.87,2,2 vehicle(s) Vehicle object at 0x7eff6d0c36a0
4,2,1,8870.4,116798.8,37.11,2,2 vehicle(s) Vehicle object at 0x7eff6d0c3760
5,2,2,6826.5,116799.2,46.70,2,2 vehicle(s) Vehicle object at 0x7eff6d0c3820
6,2,3,6697.9,116798.8,37.11,2,2 vehicle(s) Vehicle object at 0x7eff6d0c38e0
7,2,4,484.5,116799.0,41.91,2,2 vehicle(s) Vehicle object at 0x7eff6d0c39a0
8,3,1,8092.8,218510.8,36.16,2,2 vehicle(s) Vehicle object at 0x7eff6d0c3a60
9,3,2,6269.2,218511.2,46.06,2,2 vehicle(s) Vehicle object at 0x7eff6d0c3b20


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,7444.2,80328.4,33.61,1,1 vehicle(s) Vehicle object at 0x7eff6d1189d0
1,1,2,5591.8,80328.7,40.81,1,1 vehicle(s) Vehicle object at 0x7eff6d118a90
2,1,3,5784.7,80327.9,21.61,1,1 vehicle(s) Vehicle object at 0x7eff6d118af0
3,1,4,440.4,80328.0,24.01,1,1 vehicle(s) Vehicle object at 0x7eff6d118be0
4,2,1,6998.5,133064.5,33.82,1,1 vehicle(s) Vehicle object at 0x7eff6d118ca0
5,2,2,5237.8,149308.8,39.28,1,1 vehicle(s) Vehicle object at 0x7eff6d118d60
6,2,3,5406.2,133064.0,21.74,1,1 vehicle(s) Vehicle object at 0x7eff6d118e20
7,2,4,425.4,149308.2,23.57,1,1 vehicle(s) Vehicle object at 0x7eff6d118ee0
8,3,1,7028.0,229799.2,33.56,1,1 vehicle(s) Vehicle object at 0x7eff6d118fa0
9,3,2,5234.8,229799.5,41.30,1,1 vehicle(s) Vehicle object at 0x7eff6d119060


In [11]:
read_block_maxima_event_mixed_vehicles_file(froot="./", bridge_length=np.sum(bridge.L), file_format="MON")

,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,7444.2,80328.4,33.61,1,1 vehicle(s) Vehicle object at 0x7eff6d1cded0
1,1,2,5795.1,20933.9,45.45,2,2 vehicle(s) Vehicle object at 0x7eff6d1cde10
2,1,3,5784.7,80327.9,21.61,1,1 vehicle(s) Vehicle object at 0x7eff6d1cde70
3,1,4,440.4,80328.0,24.01,1,1 vehicle(s) Vehicle object at 0x7eff6d1ce1a0
4,2,1,8870.4,116798.8,37.11,2,2 vehicle(s) Vehicle object at 0x7eff6d1ce0e0
5,2,2,6826.5,116799.2,46.70,2,2 vehicle(s) Vehicle object at 0x7eff6d1ce020
6,2,3,6697.9,116798.8,37.11,2,2 vehicle(s) Vehicle object at 0x7eff6d1cdf60
7,2,4,484.5,116799.0,41.91,2,2 vehicle(s) Vehicle object at 0x7eff6d1cdea0
8,3,1,8092.8,218510.8,36.16,2,2 vehicle(s) Vehicle object at 0x7eff6d1cdde0
9,3,2,6269.2,218511.2,46.06,2,2 vehicle(s) Vehicle object at 0x7eff6d1cdd20


In [12]:
bm_summary = read_block_maxima_summary_file(froot="./", bridge_length=np.sum(bridge.L))
for i in bm_summary:
    display(i)

,index,event_1_truck,event_2_truck,event_3_truck
0,1,5784.7,5710.5,0.0
1,2,5406.2,6697.9,0.0
2,3,5372.8,6269.6,4297.4
3,4,5508.0,5618.3,0.0
4,5,5602.6,6139.3,1576.8
5,6,5621.2,6640.6,3077.3
6,7,5724.3,5949.7,1968.0
7,8,5612.2,5477.6,0.0
8,9,5836.8,5784.5,0.0
9,10,5644.6,5686.3,3706.3


,index,event_1_truck,event_2_truck,event_3_truck
0,1,440.4,394.8,0.0
1,2,425.4,484.5,0.0
2,3,408.7,442.6,323.8
3,4,420.3,419.1,0.0
4,5,422.8,457.8,126.1
5,6,420.8,493.8,246.2
6,7,437.4,433.2,157.4
7,8,445.6,409.5,0.0
8,9,440.4,420.1,0.0
9,10,428.5,415.1,261.0


,index,event_1_truck,event_2_truck,event_3_truck
0,1,5591.8,5795.1,0.0
1,2,5237.8,6826.5,0.0
2,3,5234.8,6269.2,3616.2
3,4,5330.7,5848.1,0.0
4,5,5509.7,5762.4,1825.3
5,6,5381.0,6695.8,1837.6
6,7,5545.8,5572.0,1888.3
7,8,5446.3,5771.8,0.0
8,9,5672.3,5665.3,0.0
9,10,5506.0,6333.7,2878.6


,index,event_1_truck,event_2_truck,event_3_truck
0,1,7444.2,7296.6,0.0
1,2,6998.5,8870.4,0.0
2,3,7028.0,8092.8,5632.6
3,4,7060.2,7619.5,0.0
4,5,7283.2,7642.0,2682.2
5,6,7188.2,8229.1,3518.9
6,7,7346.9,7262.2,3158.4
7,8,7264.6,7435.0,0.0
8,9,7594.4,7563.0,0.0
9,10,7296.6,7918.5,4767.3


In [13]:
from PyBTLS.py.vehicle.vehicle import Vehicle
Vehicle(path="Vehicles.txt", file_format="MON")

,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,1,1,2010,0,0,17.411,4,0,25924.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,1,1,2010,0,0,26.556,4,0,26507.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,1,1,2010,0,0,36.709,4,0,37420.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001,1,1,2010,0,0,41.751,5,0,55439.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001,1,1,2010,0,0,52.506,5,0,25512.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102596,1001,10,1,2010,23,59,21.221,4,0,38346.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102597,1001,10,1,2010,23,59,32.264,4,0,38399.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102598,1001,10,1,2010,23,59,47.763,4,0,32010.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102599,1001,10,1,2010,23,59,57.752,5,0,48898.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
